In [1]:
import os
import yfinance as yf
import quantstats as qs
import scipy.cluster.hierarchy as sch
import cvxopt as opt
import matplotlib.pyplot as plt
import shutil
import base64
import cvxpy as cp

from scipy.stats import spearmanr
from io import BytesIO
from datetime import timedelta
from IPython.display import Image
from tqdm import tqdm
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold
from cvxopt import blas, solvers
from IPython.display import display
from scipy.optimize import minimize
from matplotlib.backends.backend_svg import FigureCanvasSVG

from prep_factor import PrepFactor
from functions.utils.system import *
from functions.utils.func import *

import warnings

warnings.filterwarnings('ignore')

##### Compile 3-4 articles per stock on a daily interval. From here retrieve its embeddings and 
##### perform zero-shot classification with labels specific towards positive sentiment

##### Run a sentiment analysis on these articles and retrieve this score

##### Calculate momentum factors for these stocks and then determine which ones to buy based of these three factors

In [8]:
def get_news_fmp(tickers):
    api_key = "f913bbd3dad0c411c864c0d960a711e7"
    base_url = "https://financialmodelingprep.com/api/v3/stock_news"
    limit = 1000
    all_data = []
    for ticker in tqdm(tickers):
        page = 0
        while True:
            url = f"{base_url}?tickers={ticker}&page={page}&limit={limit}&apikey={api_key}"
            response = requests.get(url)
            data = response.json()
            if not data or 'error' in data:
                break
            all_data.extend(data)
            page += 1
    
    df = pd.DataFrame(all_data).rename(columns={'symbol':'ticker', 'publishedDate':'date'}).drop('image', axis=1)
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    df = df.set_index(['ticker', 'date']).sort_index(level=['ticker', 'date'])
    return df

In [9]:
tickers = ['AAPl', 'XLE']
news = get_news_fmp(tickers)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:37<00:00, 79.00s/it]


In [11]:
news

title  \
ticker date                                                            
AAPL   2019-02-12  SEC charges Apple's Ex-Watchdog with insider t...   
       2019-02-13               Apple's March event: What to expect?   
       2019-02-14  Here are the two industries that big hedge fun...   
       2019-02-18  Move of ex-Microsoft exec to Apple indicates c...   
       2019-02-19  Is the iPhone waterproof? Water resistance exp...   
...                                                              ...   
       2023-12-19  Apple Stock Is Being Hit by Watch, China Conce...   
       2023-12-19  Apple may be a 4T company by next year, but is...   
       2023-12-19  Hedge fund manager takes up the ‘ten surprise'...   
       2023-12-19  Apple plans rescue for $17 billion watch busin...   
       2023-12-19  Google Reaches Play Store Settlement. There's ...   

                                   site  \
ticker date                               
AAPL   2019-02-12  Bloomberg Technology   
       2019-02-13                  CNET   
       2019-02-14       CNBC Television   
       2019-02-18       CNBC Television   
       2019-02-19                  CNBC   
...                                 ...   
       2023-12-19               Barrons   
       2023-12-19            MarketBeat   
       2023-12-19          Market Watch   
       2023-12-19            TechXplore   
       2023-12-19               Barrons   

                                                                text  \
ticker date                                                            
AAPL   2019-02-12  Apple Inc. former top lawyer Gene Daniel Levof...   
       2019-02-13  AirPods, iPads, news about its streaming servi...   
       2019-02-14  It's the time of the season when investors are...   
       2019-02-18  Tripp Mickle, WSJ, and Christina Farr, CNBC te...   
       2019-02-19  Are iPhones waterproof? A few years ago compan...   
...                                                              ...   
       2023-12-19  Apple stock is struggling to break through the...   
       2023-12-19  Apple Inc. NASDAQ: AAPL is one of the most hot...   
       2023-12-19  Byron Wien, a respected market strategist at M...   
       2023-12-19  Apple Inc., just days away from a U.S. ban of ...   
       2023-12-19  Google-parent Alphabet has agreed to a $700 mi...   

                                                                 url  
ticker date                                                           
AAPL   2019-02-12        https://www.youtube.com/watch?v=jM4dWUOjqHg  
       2019-02-13        https://www.youtube.com/watch?v=lnn7eHpu5Dw  
       2019-02-14   https://www.youtube.com/watch?v=GHRwqcUUJks&t=0s  
       2019-02-18        https://www.youtube.com/watch?v=McK7QWqCxy8  
       2019-02-19        https://www.youtube.com/watch?v=4dyL6hMZvWQ  
...                                                              ...  
       2023-12-19  https://www.barrons.com/articles/apple-stock-b...  
       2023-12-19  https://www.marketbeat.com/originals/apple-may...  
       2023-12-19  https://www.marketwatch.com/story/hedge-fund-m...  
       2023-12-19  https://techxplore.com/news/2023-12-apple-bill...  
       2023-12-19  https://www.barrons.com/articles/google-alphab...  

[22598 rows x 4 columns]

In [ ]:
# Shift all news one date forward to avoid lookahead bias